In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
import tensorflow.compat.v1 as tf
from tensorflow.compat.v1.distributions import Normal, kl_divergence
import numpy as np
import matplotlib.pyplot as plt
import os
import argparse
import time
from scipy.special import logsumexp
tf.disable_eager_execution()

In [ ]:
# Bayes Dense Layer

def bayes_dense(x, num_units, name='dense', gamma=1.0, activation=None):
    with tf.variable_scope(name, reuse=tf.AUTO_REUSE):
        W_mu = tf.get_variable('W_mu', [x.shape[1], num_units])
        W_rho = tf.nn.softplus(
                tf.get_variable('W_rho', [x.shape[1], num_units],
                    initializer=tf.random_uniform_initializer(-3., -2.)))
        b_mu = tf.get_variable('b_mu', [num_units],
                initializer=tf.zeros_initializer())
        b_rho = tf.nn.softplus(
                tf.get_variable('b_rho', [num_units],
                    initializer=tf.random_uniform_initializer(-3., -2.)))

    xW_mean = tf.matmul(x, W_mu)
    xW_std = tf.sqrt(tf.matmul(tf.square(x), tf.square(W_rho)) + 1e-6)
    xW = xW_mean + xW_std*tf.random.normal(tf.shape(xW_mean))
    b = b_mu + b_rho * tf.random.normal(b_mu.shape)

    x = xW + b
    if activation == 'relu':
        x = tf.nn.relu(x)

    # kl divergence
    kld_W = tf.reduce_sum(kl_divergence(Normal(W_mu, W_rho), Normal(0., gamma)))
    kld_b = tf.reduce_sum(kl_divergence(Normal(b_mu, b_rho), Normal(0., gamma)))
    kld = kld_W + kld_b

    return x, kld

In [ ]:
# generate data

def func(x, noise):
    eps = noise * np.random.randn(*x.shape)
    return x + eps + np.sin(4*(x + eps)) + np.sin(13*(x + eps))

num_train = 50
num_test = 20
np.random.seed(42)
x_train_np = np.concatenate([
    0.5 * np.random.rand(num_train//2, 1),
    0.8 + 0.2*np.random.rand(num_train//2, 1)], 0)
y_train_np = func(x_train_np, 0.03)
x_test_np = np.random.rand(num_test, 1)
y_test_np = func(x_test_np, 0.03)
np.random.seed(int(time.time()))

# Problem 1, 2: Fill in the missing parts.

In [ ]:
# Training

kl_coeff = 0.01
num_steps = 10000
num_samples = 50

x = tf.placeholder(tf.float32, shape=[None, 1])
y = tf.placeholder(tf.float32, shape=[None, 1])

# define network
# BayesDense(784, 100) - ReLU
# BayesDense(100, 100) - ReLU
# BayesDense(100, 2)
#### Problem 1 ####
out, kld1 = 
out, kld2 = 
out, kld3 = 
###################
mu, sigma = tf.split(out, 2, axis=-1)
sigma = tf.nn.softplus(sigma)

# log-likelihood
ll = tf.reduce_mean(Normal(mu, sigma).log_prob(y))
# kl-divergence
kld = kl_coeff * (kld1 + kld2 + kld3) / np.float32(num_train)

lr = tf.placeholder(tf.float32)
train_op = tf.train.AdamOptimizer(lr).minimize(#### Problem 2 ####)

def get_lr(t):
    if t < 0.25 * num_steps:
        return 0.01
    elif t < 0.5 * num_steps:
        return 0.1 * 0.01
    else:
        return 0.01 * 0.01

# training loop
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for t in range(num_steps):
    
    sess.run(train_op, {x:x_train_np, y:y_train_np, lr:get_lr(t)})

    if (t+1)% 100 == 0:
        train_ll, train_kld = sess.run([ll, kld], {x:x_train_np, y:y_train_np})
        print('step %d train ll %.4f train kld %.4f' % (t+1, train_ll, train_kld))

# You can check your code by running the below cell

In [ ]:
# Test & Visualization

x_np = np.linspace(0.0, 1.0, 100)[...,None]
y_true_np = func(x_np, 0.0)

mu_np_list, sigma_np_list = [], []
ll_np_list = []
for i in range(num_samples):
    mu_np, sigma_np = sess.run([mu, sigma], {x:x_np})
    mu_np, sigma_np = mu_np.squeeze(), sigma_np.squeeze()
    ll_np = sess.run(Normal(mu, sigma).log_prob(y), {x:x_test_np, y:y_test_np})
    mu_np_list.append(mu_np)
    sigma_np_list.append(sigma_np)
    ll_np_list.append(ll_np)

ll_np_list = np.stack(ll_np_list, 0).squeeze(-1)
print('test ll: %.4f' % (logsumexp(ll_np_list, 0) - np.log(num_samples)).mean())

x_np = x_np.squeeze()
y_true_np = y_true_np.squeeze()
x_train_np = x_train_np.squeeze()
y_train_np = y_train_np.squeeze()
x_test_np = x_test_np.squeeze()
y_test_np = y_test_np.squeeze()

plt.figure()
for i in range(num_samples):
    upper = mu_np_list[i].squeeze() + sigma_np_list[i].squeeze()
    lower = mu_np_list[i].squeeze() - sigma_np_list[i].squeeze()
    plt.fill_between(x_np, lower, upper, alpha=0.01, color='skyblue')
    plt.plot(x_np, mu_np_list[i].squeeze(), color='blue', alpha=0.05)

plt.plot(x_train_np, y_train_np, 'r.', label='train')
plt.plot(x_test_np, y_test_np, 'b*', label='test')
plt.plot(x_np, y_true_np, 'k--', label='true')
plt.legend()
plt.show()